In [1]:
import os
import warnings

import numpy as np
import pandas as pd

from fbprophet import Prophet

%load_ext memory_profiler

Importing plotly failed. Interactive plots will not work.


In [2]:
test_size = 7
datasets = [{
    'name': 'casos_confirmados',
    'path': 'data/covid/casos_confirmados.csv'
}, {
    'name': 'sinteticos',
    'path': 'data/sinteticos/sinteticos.csv'
}, {
    'name': 'temperaturas',
    'path': 'data/temperatura_minima_diaria/temperaturas.csv'
}]
metrics = ['rmse', 'mape', 'mae', 'mpe']

In [3]:
parameters = {
    'growth': 'linear',
    'changepoints': None,
    'n_changepoints': 25,
    'changepoint_range': 0.8,
    'yearly_seasonality': 'auto',
    'weekly_seasonality': 'auto',
    'daily_seasonality': 'auto',
    'holidays': None,
    'seasonality_mode': 'additive',
    'seasonality_prior_scale': 10.0,
    'holidays_prior_scale': 10.0,
    'changepoint_prior_scale': 0.05,
    'mcmc_samples': 0,
    'interval_width': 0.80,
    'uncertainty_samples': 1000,
    'stan_backend': None
}

In [4]:
warnings.filterwarnings('ignore')
for dataset in datasets:
    ts = pd.read_csv(dataset['path'], sep=';')
    ts.columns = ['ds', 'y']

    if dataset['name'] == 'casos_confirmados':
        parameters['growth'] = 'logistic'
        ts['floor'] = 0
        ts['cap'] = 2000
        parameters['daily_seasonality'] = False
        parameters['weekly_seasonality'] = True
        parameters['yearly_seasonality'] = False
    elif dataset['name'] == 'temperaturas':
        parameters['growth'] = 'logistic'
        ts['cap'] = 30
        ts['floor'] = -5
        parameters['daily_seasonality'] = True
        parameters['weekly_seasonality'] = False
        parameters['yearly_seasonality'] = True
    else:
        ts['cap'] = None
        ts['floor'] = None
        parameters['daily_seasonality'] = False
        parameters['weekly_seasonality'] = True
        parameters['yearly_seasonality'] = False

    grid_results_df = pd.read_csv(os.path.join('results/prophet', f"{dataset['name']}_grid_results.csv"), sep=';')

    for metric in metrics:
        print(f"Dataset: {dataset['name']} | Optimization Metric: {metric.upper()}")

        best = grid_results_df[grid_results_df[metric].abs().eq(grid_results_df[metric].abs().min())].iloc[0]
        params = best.iloc[:-4].to_dict()
        params = {k: v for k, v in params.items() if not (np.isnan(v) if type(v) == np.float64 else False)}
        parameters.update(params)
        print(f"Parameters: {' '.join(['{}={}'.format(pkey, pvalue) for pkey, pvalue in parameters.items()])}")
        try:
            %timeit -n10 Prophet(**parameters).fit(ts)  # Fit model with given params
            m = Prophet(**parameters).fit(ts)  # Fit model with given params
            future = m.make_future_dataframe(periods=test_size)
            future['floor'] = ts.loc[:, 'floor'].iloc[0]
            future['cap'] = ts.loc[:, 'cap'].iloc[0]
            %timeit -n100 forecast = m.predict(future)
        except RuntimeError:
            print(f'[ERROR] Error with following hyperparameters: {parameters}')

Dataset: casos_confirmados | Optimization Metric: RMSE
Parameters: growth=logistic changepoints=None n_changepoints=15 changepoint_range=0.8 yearly_seasonality=False weekly_seasonality=True daily_seasonality=False holidays=None seasonality_mode=additive seasonality_prior_scale=0.01 holidays_prior_scale=10.0 changepoint_prior_scale=0.001 mcmc_samples=0 interval_width=0.8 uncertainty_samples=1000 stan_backend=None
56.5 ms ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.26 s ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
Dataset: casos_confirmados | Optimization Metric: MAPE
Parameters: growth=logistic changepoints=None n_changepoints=15 changepoint_range=0.8 yearly_seasonality=False weekly_seasonality=True daily_seasonality=False holidays=None seasonality_mode=additive seasonality_prior_scale=1.0 holidays_prior_scale=10.0 changepoint_prior_scale=0.001 mcmc_samples=0 interval_width=0.8 uncertainty_samples=1000 stan_backend=None
46 ms ± 63.1 µs per loop (m